In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data() # we don't need the labels
x_train = x_train.astype('float32') / 255. # normalise
x_test = x_test.astype('float32') / 255.

print(x_train.shape) # shape of input
print(x_test.shape)

x_train = x_train.reshape(( x_train.shape[0], np.prod(x_train.shape[1:]))) #flatten
x_test = x_test.reshape((x_test.shape[0], np.prod(x_test.shape[1:])))

print(x_train.shape)
print(x_test.shape)

In [ ]:
image_dim =  784 # this is the size of our input image, 784
encoding_dim = 32 # this is the length of our encoded items.Compression of factor=784/32=24.5

In [ ]:
input_image = Input(shape=(image_dim, )) # this is our input placeholder

encoded_image = Dense(encoding_dim, activation='relu',
                      activity_regularizer=regularizers.l1(10e-5))(input_image)# "encoded" is the encoded representation of the input
encoder = Model(input_image, encoded_image)

decoded_image = Dense(image_dim, activation='sigmoid')(encoded_image)# "decoded" is the lossy reconstruction of the input

autoencoder = Model(input_image, decoded_image) # this model maps an input to its reconstruction

In [ ]:

encoded_input = Input(shape=(encoding_dim,))# create a placeholder for an encoded (32-dimensional) input

decoder_layer = autoencoder.layers[-1]# retrieve the last layer of the autoencoder model

decoder = Model(encoded_input, decoder_layer(encoded_input))# create the decoder model

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
checkpointer1 = ModelCheckpoint(filepath= 'model.weights.best.hdf5' , verbose =2, save_best_only =  True)
checkpointer2 = EarlyStopping(monitor='val_loss',
                              min_delta=0.0005,
                              patience=2,
                              verbose=2, mode='auto')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=500,
                batch_size=256,callbacks=[checkpointer1], verbose=2,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# encode and decode some items
# note that we take them from the *test* set

autoencoder.load_weights('model.weights.best.hdf5' )
encoded_images = encoder.predict(x_test)
decoded_images = decoder.predict(encoded_images)

In [ ]:
number_of_items = 12  # how many tems we will display
plt.figure(figsize=(20, 4))
for i in range(number_of_items):
    # display items before compression
    graph = plt.subplot(2, number_of_items, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)

    # display items after decompression
    graph = plt.subplot(2, number_of_items, i + 1 + number_of_items)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)
plt.show()